# Mothers’ Lifestyle Characteristics Impact on Infant Birth Weight

Team B1: Ayingfu (#82951880), Bryant Hartono (#66162710), Jeremy Davies (#74883935), Jingxuan Ma (#49183288)

## 1. Introduction

#### 1.1 Data Source

The dataset for this project is the “birthwt” dataset provided with the Support Functions and Datasets for Venables and Ripley's MASS package in R.

Relevant citation for the MASS package:  
Venables WN, Ripley BD (2002). Modern Applied Statistics with S, Fourth edition. Springer, New York. ISBN 0-387-95457-0, https://www.stats.ox.ac.uk/pub/MASS4/.

#### 1.2 Dataset Description

**Dataset detail**  
The dataset consists of 189 observations with 10 columns (refer to column details explained below). Data was collected at Baystate Medical Center, Springfield, Massachusetts during 1986, with no specific setting. Study subjects were 189 mothers, 59 of which had low birth weight babies and 130 of which had normal birth weight babies. Characteristics of the mother’s lifestyle and health factors were collected during the pregnancy and the weight of the newborn infant was recorded at birth.

**Response variable**  
Neonate birth weight (‘bwt’), measured in grams.

**Potential explanatory variables**
1. Continuous variables
- age: Mother’s age (years)
- lwt: Mother's weight (pounds) at last menstrual period
- ptl: Number of previous premature labors
- ftv: Number of physician visits during the first trimester
2. Categorical variables
- race: Mother's race ( “1” = white, “2” = black, “3” = other) 
- smoke: Smoking status during the pregnancy. (“1”=Yes,”0”= No)
- ht: History of hypertension. (“1”=Yes,”0”= No)
- ui: presence of uterine irritability. (“1”=Yes,”0”= No)

#### 1.3 Research Question / Motivation

There is some evidence (and some speculation) to suggest that heavier babies have lower infant mortality rates and perhaps also lower risk of cardiovascular disease later in life. Therefore, our group is interested in studying the factors that explain an infant’s birth weight. In particular, we are interested in whether the weight of the mother, the mother’s race, the smoking status of the mother, among other variables, affect the infant birth weight.  
  
If we can identify the contributing factors and reasonably predict a baby’s neonate birth weight, this would enable better preparation of healthcare at birth. For example, if we develop a model that predicts a baby will have a low birth weight, then the obstetrician could recommend for the mother to have the birth at a dedicated neonatal ward, where there are more specialized resources to ensure a healthy birth.

## 2. Analysis

## 3. Conclusion